In [246]:
import findspark
findspark.init()

In [247]:
import pyspark
from pyspark.sql import functions

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
sc = pyspark.SparkContext(appName="bootstrap")

In [250]:
sqlCtx = pyspark.SQLContext(sc)

In [2]:
# @user credentials

user='leon.zhao@offerupnow.com'
password='Steel24ly@offerup'
account='offerup'

In [3]:
import snowflake.connector
import pandas as pd
import numpy as np



class SnowflakeSession:

    def __init__(self, user, password, account):

        # Connect and get cursor
        print("> Connecting to Snowflake")
        self.ctx = snowflake.connector.connect(user=user,
                                                password=password,
                                                account=account, )
        self.cs = self.ctx.cursor()

        # Set up cursor
        self.cs.execute("use warehouse analytics_wh;")
        self.cs.execute("use role marketing;")
        self.cs.execute("Alter session set timezone = 'America/Los_Angeles';") # for TZ aware fields

    @staticmethod
    def parse_query_file(sql_query_file):
        """Parse SQL file and return query as string."""
        
        # Reading of the SQL statement from sql_query.txt file
        print("> Reading SQL statement from file")
        fd = open(sql_query_file, 'r')
        sqlFile = fd.read()
        fd.close()

        # Extraction of the SQL statement (split on ';')
        # Get only the first statement.
        sql_sentence_list = sqlFile.split(';')
        sql_query = [x for x in sql_sentence_list if x][0]

        return sql_query

    def execute_query(self, qry):
        print("> Querying database with:")
        print(qry)
        print("\n> Fetching data ...")
        self.cs.execute(qry)
        colnames = [x[0] for x in self.cs.description]
        rows = self.cs.fetchall()
        return rows, colnames

    @staticmethod
    def result_to_df(rows, colnames):
        print("> Saving to dataframe ...")
        df =  pd.DataFrame(rows, columns=colnames)
        df.replace({r'\r': ''}, regex=True, inplace=True)
        return df


    @staticmethod
    def save_df_to_csv(df, fname):
        print("\n> Saving to file ...")
        if '.csv' in fname:
            separator = ','
        else:
            separator = '\t'
        df.to_csv(fname, index=False, sep=separator, header=True, encoding='utf-8')

    def close_session(self):
        print("\n> Closing session ...")
        self.cs.close()
        self.ctx.close()
        
s = SnowflakeSession(user, password, account)

> Connecting to Snowflake


# Adjust Dataset for bootstrap

In [4]:
data = s.parse_query_file('/Users/leon.zhao/Desktop/SQL/simplification_test_summary')
print(data)
rows, colnames = s.execute_query(data)

> Reading SQL statement from file
select *
from ou_users.leon.sa_simplification_summary

> Querying database with:
select *
from ou_users.leon.sa_simplification_summary


> Fetching data ...


In [5]:
df = pd.DataFrame(data = rows, columns = colnames)
df.head()

,PLAT,USER_ID,VARIANT_ID,FIRST_EXPOSURE_TS,VARIANT_NAME,BUMP_REVENUE,PROMOTION_REVENUE,SUB_REVENUE,BUMPS,ITEM_PROMOTIONS,SUB_PURCHASES,SA_1D_REVENUE,SA_3D_REVENUE,SA_7D_REVENUE,SA_14D_REVENUE,PRICE_1DAY_PURCHASES,PRICE_3DAY_PURCHASES,PRICE_7DAY_PURCHASES,PRICE_14DAY_PURCHASES
0,AD,3517258,"""5123000743493632""",2019-02-17 07:17:10,Subs_1d_3d,0.0,0.00,0.0,0,0,0,0.0,0.00,0.0,0.0,0,0,0,0
1,AD,56030756,"""5123000743493632""",2019-02-26 02:11:56,Subs_1d_3d,0.0,0.00,0.0,0,0,1,0.0,0.00,0.0,0.0,0,0,0,0
2,AD,41697759,"""5123000743493632""",2019-02-10 01:43:04,Subs_1d_3d,0.0,7.99,0.0,0,1,0,0.0,7.99,0.0,0.0,0,1,0,0
3,AD,34729619,"""5123000743493632""",2019-02-24 18:59:59,Subs_1d_3d,0.0,0.00,0.0,0,0,0,0.0,0.00,0.0,0.0,0,0,0,0
4,AD,19660207,"""5123000743493632""",2019-02-11 01:53:37,Subs_1d_3d,0.0,0.00,0.0,0,0,0,0.0,0.00,0.0,0.0,0,0,0,0


In [6]:
num_iterations = 10000

In [7]:
df1 = df.sort_values('BUMP_REVENUE',ascending = False)
df = df1[df1.BUMP_REVENUE <= 500]
df.head(10)

,PLAT,USER_ID,VARIANT_ID,FIRST_EXPOSURE_TS,VARIANT_NAME,BUMP_REVENUE,PROMOTION_REVENUE,SUB_REVENUE,BUMPS,ITEM_PROMOTIONS,SUB_PURCHASES,SA_1D_REVENUE,SA_3D_REVENUE,SA_7D_REVENUE,SA_14D_REVENUE,PRICE_1DAY_PURCHASES,PRICE_3DAY_PURCHASES,PRICE_7DAY_PURCHASES,PRICE_14DAY_PURCHASES
123704,iOS,43254552,"""5508595393626112""",2019-02-06 02:08:37,Control,469.65,0.00,0.0,235,0,0,0.0,0.00,0.0,0.0,0,0,0,0
463336,iOS,31745425,"""5751559409369088""",2019-02-09 16:56:16,Subs_bump_3d,320.39,0.00,0.0,161,0,0,0.0,0.00,0.0,0.0,0,0,0,0
74298,AD,6995374,"""5685950696914944""",2019-02-08 12:34:03,Subs_bump_3d,304.47,0.00,0.0,153,0,0,0.0,0.00,0.0,0.0,0,0,0,0
58006,iOS,31515662,"""5751559409369088""",2019-02-05 16:17:33,Subs_bump_3d,292.54,7.99,0.0,146,1,0,0.0,7.99,0.0,0.0,0,1,0,0
469543,iOS,52962412,"""5508595393626112""",2019-02-06 01:05:56,Control,291.27,335.58,0.0,73,42,0,0.0,335.58,0.0,0.0,0,42,0,0
291481,iOS,55650085,"""5751559409369088""",2019-02-17 04:59:32,Subs_bump_3d,286.65,423.47,0.0,135,53,0,0.0,423.47,0.0,0.0,0,53,0,0
533757,iOS,7758098,"""5751559409369088""",2019-02-06 15:13:56,Subs_bump_3d,223.44,15.98,0.0,56,2,0,0.0,15.98,0.0,0.0,0,2,0,0
166487,iOS,49701946,"""5508595393626112""",2019-02-09 01:18:20,Control,201.02,0.00,0.0,98,0,0,0.0,0.00,0.0,0.0,0,0,0,0
227525,iOS,1348126,"""5508595393626112""",2019-02-06 22:14:35,Control,200.99,0.00,0.0,101,0,0,0.0,0.00,0.0,0.0,0,0,0,0
281091,iOS,13037723,"""5229755043807232""",2019-02-06 02:03:16,Subs_1d_3d_7d_14d,195.02,0.00,0.0,98,0,0,0.0,0.00,0.0,0.0,0,0,0,0


In [8]:
AD = df.loc[df['PLAT'] == 'AD']
ad_rev = AD.loc[:,['PLAT','VARIANT_NAME','BUMP_REVENUE','PROMOTION_REVENUE','SUB_REVENUE']]
ad_rev_zero = AD.loc[:,['PLAT','VARIANT_NAME','BUMP_REVENUE','PROMOTION_REVENUE','SUB_REVENUE']]
ad_rev = ad_rev[(ad_rev.BUMP_REVENUE != 0.0) | (ad_rev.PROMOTION_REVENUE != 0.0) | (ad_rev.SUB_REVENUE != 0.0)]
ad_rev['IAP_REVENUE'] = ad_rev.iloc[:,1:].sum(axis=1)
ad_rev_zero['IAP_REVENUE'] = ad_rev_zero.iloc[:,1:].sum(axis=1)


ads = {}
for name in ['Control', 'Subs_1d_3d', 'Subs_bump_3d', 'Subs_1d_3d_7d_14d']:
    ad = ad_rev.loc[ad_rev['VARIANT_NAME'] == name]
    ad_iap_rev = ad.iloc[:,-1]
    ads[name] = ad_iap_rev
    print('ad', name, ':', len(ads[name]))
    
    
ad_zero = {}
for name in ['Control', 'Subs_1d_3d', 'Subs_bump_3d', 'Subs_1d_3d_7d_14d']:
    adz = ad_rev_zero.loc[ad_rev_zero['VARIANT_NAME'] == name]
    adz_iap_rev = adz.iloc[:,-1]
    ad_zero[name] = adz_iap_rev
    print('ad_zero', name, ':', len(ad_zero[name]))

    
    
ad_cvt = {}
for name in ['Control', 'Subs_1d_3d', 'Subs_bump_3d', 'Subs_1d_3d_7d_14d']:
    p = len(ads[name])/len(ad_zero[name])
    print(p)
    ad_cvt[name] = np.random.binomial(len(ad_zero[name]), p) 
print(ad_cvt)

ad Control : 2436
ad Subs_1d_3d : 2134
ad Subs_bump_3d : 2492
ad Subs_1d_3d_7d_14d : 2181
ad_zero Control : 44366
ad_zero Subs_1d_3d : 44890
ad_zero Subs_bump_3d : 44573
ad_zero Subs_1d_3d_7d_14d : 44404
0.05490691069738088
0.04753842726665181
0.055908285284813675
0.04911719664895055
{'Control': 2413, 'Subs_1d_3d': 2166, 'Subs_bump_3d': 2422, 'Subs_1d_3d_7d_14d': 2223}


In [9]:
for variant_name, variant_data in ads.items():
    print(variant_name)
    print(variant_data.describe())

Control
count    2436.000000
mean       10.814290
std        13.425165
min         1.990000
25%         1.990000
50%         7.990000
75%        19.990000
max       207.890000
Name: IAP_REVENUE, dtype: float64
Subs_1d_3d
count    2134.000000
mean       11.296659
std        11.312205
min         1.990000
25%         2.990000
50%         7.990000
75%        19.990000
max       291.280000
Name: IAP_REVENUE, dtype: float64
Subs_bump_3d
count    2492.000000
mean       10.422833
std        15.233118
min         1.990000
25%         1.990000
50%         7.980000
75%        15.980000
max       304.470000
Name: IAP_REVENUE, dtype: float64
Subs_1d_3d_7d_14d
count    2181.000000
mean       12.866016
std        16.772935
min         1.990000
25%         2.990000
50%         7.990000
75%        19.990000
max       497.490000
Name: IAP_REVENUE, dtype: float64


In [10]:
dic = {'ad_control_iap_rev':ads['Control'],
      'ad_Subs_1d_3d_iap_rev':ads['Subs_1d_3d'],
      'ad_Subs_bump_3d_iap_rev':ads['Subs_bump_3d'],
      'ad_Subs_1d_3d_7d_14d_iap_rev':ads['Subs_1d_3d_7d_14d']}
for i, j in dic.items():
    plt.hist(j, bins=np.arange(min(j)+1, max(j) + 1, 1), label=i, density = False)
    plt.title(i)
    plt.legend()
    plt.show()

NameError: name 'plt' is not defined

In [11]:
iOS = df.loc[df['PLAT'] == 'iOS']
ios_rev = iOS.loc[:,['PLAT','VARIANT_NAME','BUMP_REVENUE','PROMOTION_REVENUE','SUB_REVENUE']]
# ios_rev = ios_rev[(ios_rev.BUMP_REVENUE != 0.0) | (ios_rev.PROMOTION_REVENUE != 0.0) | (ios_rev.SUB_REVENUE != 0.0)]
ios_rev['IAP_REVENUE'] = ios_rev.iloc[:,1:].sum(axis=1)
ios_rev.head(10)


ls = ['Control','Subs_1d_3d','Subs_bump_3d','Subs_1d_3d_7d_14d']
    


ios_control = ios_rev.loc[ios_rev['VARIANT_NAME'] == 'Control']
ios_Subs_1d_3d = ios_rev.loc[ios_rev['VARIANT_NAME'] == 'Subs_1d_3d']
ios_Subs_bump_3d = ios_rev.loc[ios_rev['VARIANT_NAME'] == 'Subs_bump_3d']
ios_Subs_1d_3d_7d_14d = ios_rev.loc[ios_rev['VARIANT_NAME'] == 'Subs_1d_3d_7d_14d']



ios_control_iap_rev = ios_control.iloc[:,-1]
ios_Subs_1d_3d_iap_rev = ios_Subs_1d_3d.iloc[:,-1]
ios_Subs_bump_3d_iap_rev = ios_Subs_bump_3d.iloc[:,-1]
ios_Subs_1d_3d_7d_14d_iap_rev = ios_Subs_1d_3d_7d_14d.iloc[:,-1]

In [12]:
for i in [ios_control_iap_rev,
          ios_Subs_1d_3d_iap_rev,
          ios_Subs_bump_3d_iap_rev,
          ios_Subs_1d_3d_7d_14d_iap_rev]:
    print(i.describe())

count    93672.000000
mean         0.921432
std          6.244083
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        626.850000
Name: IAP_REVENUE, dtype: float64
count    93861.000000
mean         0.880765
std          5.230918
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        455.430000
Name: IAP_REVENUE, dtype: float64
count    93683.000000
mean         0.880844
std          6.398738
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        710.120000
Name: IAP_REVENUE, dtype: float64
count    93468.000000
mean         0.917731
std          5.588814
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max        506.510000
Name: IAP_REVENUE, dtype: float64


In [13]:
dic = {'ios_control_iap_rev':ios_control_iap_rev,
      'ios_Subs_1d_3d_iap_rev':ios_Subs_1d_3d_iap_rev,
      'ios_Subs_bump_3d_iap_rev':ios_Subs_bump_3d_iap_rev,
      'ios_Subs_1d_3d_7d_14d_iap_rev':ios_Subs_1d_3d_7d_14d_iap_rev}
for i, j in dic.items():
    plt.hist(j, bins=np.arange(min(j)+1, max(j) + 1, 1), label=i, density = False)
    plt.title(i)
    plt.legend()
    plt.show()
    
    

NameError: name 'plt' is not defined

# Bootstrap with Numpy

In [14]:
# ctrl = ads['ad_Control']
# test = ads['ad_Subs_1d_3d']
# test = ads['ad_Subs_bump_3d']
# test = ads['ad_Subs_1d_3d_7d_14d']




ctrl = ios_control_iap_rev
# test = ios_Subs_1d_3d_iap_rev
# test = ios_Subs_bump_3d_iap_rev
test = ios_Subs_1d_3d_7d_14d_iap_rev


ad_cvt = {}
p = len(ad_rev['IAP_REVENUE'])/len(ad_rev_zero['IAP_REVENUE'])
for name in ['Control', 'Subs_1d_3d', 'Subs_bump_3d', 'Subs_1d_3d_7d_14d']:
    ad_cvt[name] = np.random.binomial(len(ad_zero[name]), p, 1) 
#     ad_cvt[name] = ad_cvt[name]/len(ad_rev_zero.loc[ad_rev_zero['VARIANT_NAME'] == name])
    print(ad_cvt)


bootstrapped_means_ctrl = []
bootstrapped_means_test = []
for i in range(num_iterations):
    data_bootstrap_ctrl = np.random.choice(ctrl, size=ctrl.shape[0], replace=True)
    data_bootstrap_test = np.random.choice(test, size=test.shape[0], replace=True)
    ctrl_mean = np.mean(data_bootstrap_ctrl)
    test_mean = np.mean(data_bootstrap_test)
    bootstrapped_means_ctrl.append(ctrl_mean)
    bootstrapped_means_test.append(test_mean)
bootstrapped_means_ctrl = np.array(bootstrapped_means_ctrl)
bootstrapped_means_test = np.array(bootstrapped_means_test)
bootstrapped_means_diff = bootstrapped_means_test - bootstrapped_means_ctrl

{'Control': array([2271])}
{'Control': array([2271]), 'Subs_1d_3d': array([2289])}
{'Control': array([2271]), 'Subs_1d_3d': array([2289]), 'Subs_bump_3d': array([2325])}
{'Control': array([2271]), 'Subs_1d_3d': array([2289]), 'Subs_bump_3d': array([2325]), 'Subs_1d_3d_7d_14d': array([2319])}


In [15]:
bootstrapped_means_diff

array([-0.01455922,  0.01628642,  0.01298921, ...,  0.01914518,
        0.01781066,  0.03477662])

In [16]:
alpha=0.05
low = np.percentile(bootstrapped_means_diff, 100 * (alpha / 2.))
val = np.percentile(bootstrapped_means_diff, 50)
high = np.percentile(bootstrapped_means_diff, 100 * (1 - alpha / 2.))
x_bar = np.mean(test) - np.mean(ctrl)
# print(ctrl.describe(), test.describe())
print('{}\t({}, {})'.format(val,low,high))
print('p-value: {}'.format(min(2*np.sum(bootstrapped_means_diff < 0)/bootstrapped_means_diff.shape[0],
                               2*np.sum(bootstrapped_means_diff > 0)/bootstrapped_means_diff.shape[0])))

-0.003786711977154855	(-0.0578286653345242, 0.04962371453763348)
p-value: 0.8922


In [17]:
bootstrapped_means_diff.shape[0]

10000

# Bootsrap with PySpark

each data frame includes all users and metrics from a given experiment and variant

this assumes a dense dataset where most users have non-zero values for the metric (e.g. clicks)

in the case of a sparse metric (e.g. shipment) see the trick below to speed up sampling

In [ ]:
# ctrl = ads['ad_Control']
# test = ads['ad_Subs_1d_3d']
# test = ads['ad_Subs_bump_3d']
# test = ads['ad_Subs_1d_3d_7d_14d']




ctrl = ios_control_iap_rev
# test = ios_Subs_1d_3d_iap_rev
test = ios_Subs_bump_3d_iap_rev
# test = ios_Subs_1d_3d_7d_14d_iap_rev

sdf_ctrl = sqlCtx.createDataFrame(pd.DataFrame({'metric_1': ctrl, 'metric_2': 2*ctrl}))
sdf_test = sqlCtx.createDataFrame(pd.DataFrame({'metric_1': test, 'metric_2': 2*test}))

In [ ]:
bootstrapped_means_diff = []
# TODO: this for loop could be parallelized
for i in range(num_iterations):
    bootstrap_ctrl = sdf_ctrl.sample(withReplacement=True, fraction=1.0)
    bootstrap_test = sdf_test.sample(withReplacement=True, fraction=1.0)
    ctrl_mean = np.array(bootstrap_ctrl.select(*[functions.mean(col) for col in bootstrap_ctrl.columns]).collect())
    test_mean = np.array(bootstrap_test.select(*[functions.mean(col) for col in bootstrap_test.columns]).collect())
    means_diff = test_mean - ctrl_mean
    bootstrapped_means_diff.append(means_diff)
bootstrapped_means_diff = np.array(bootstrapped_means_diff)

two tailed stat sig test, null hypothesis of no difference for metric 1

In [ ]:
low = np.percentile(bootstrapped_means_diff[0], 100 * (alpha / 2.))
val = np.percentile(bootstrapped_means_diff[0], 50)
high = np.percentile(bootstrapped_means_diff[0], 100 * (1 - alpha / 2.))
print('{}\t({}, {})'.format(val,low,high))
print('p-value: {}'.format(min(2*np.sum(bootstrapped_means_diff[0] < 0)/bootstrapped_means_diff.shape[0],
                               2*np.sum(bootstrapped_means_diff[0] > 0)/bootstrapped_means_diff.shape[0])))

# Notes / References

**note on case of sparse data from facebook implementation --**

in the sparse case we dont actually need to bootstrap
the full sparse array since most values are 0
instead for each bootstrap iteration we:
   1. generate B number of non-zero entries to sample from the binomial distribution with n = all users buckets and p = proportion of users with non-zero entries
   2. resample with replacement the non-zero entries from values B times
   3. create a new sparse array with the B resamples, zero otherwise

**references**
* http://faculty.psy.ohio-state.edu/myung/personal/course/826/bootstrap_hypo.pdf
* https://ocw.mit.edu/courses/mathematics/18-05-introduction-to-probability-and-statistics-spring-2014/readings/MIT18_05S14_Reading24.pdf
* https://github.com/facebookincubator/bootstrapped/blob/master/bootstrapped/bootstrap.py
* https://fullstackml.com/how-to-check-hypotheses-with-bootstrap-and-apache-spark-cd750775286a